In [154]:
"""Docstring"""

import csv
import collections
import re

#Read Data
def readdata():
    # This function reads data from the original dataset
    with open('all_stocks_5yr.csv') as fp:
        reader = csv.reader(fp)
        data = list(reader)
    return data

def assetlist():
    # This function returns a set of tickers of S&P500 stocks
    n = readdata()
    k = [ i[6] for i in n]
    return set(k)

assetlist = assetlist() # A list of all stocks in the dataset for future reference
dataset = readdata() # The original dataset for future reference
data = dataset[1:] # The original dataset without header for future reference


In [100]:
import data_analysis

Missing Values
Missing values for: "date"
Empty DataFrame
Columns: [date, open, high, low, close, volume, Name]
Index: []


Missing values for: "open"
              date  open   high    low     close   volume  Name
82949   2017-07-26   NaN    NaN    NaN   69.0842        3   BHF
165734  2015-07-17   NaN  88.76  88.24   88.7200  2056819   DHR
165857  2016-01-12   NaN    NaN    NaN   88.5500        0   DHR
205076  2015-07-17   NaN  48.49  47.85   47.9200  1246786    ES
239832  2016-07-01   NaN    NaN    NaN   49.5400        0   FTV
434379  2015-07-17   NaN  47.31  46.83   46.9900  1229513     O
434502  2016-01-12   NaN    NaN    NaN   52.4300        0     O
478594  2015-06-09   NaN    NaN    NaN  526.0900    12135  REGN
558213  2016-04-07   NaN    NaN    NaN   41.5600        0    UA
581906  2015-05-12   NaN    NaN    NaN  124.0800   569747  VRTX
598236  2015-06-26   NaN    NaN    NaN   61.9000      100   WRK


Missing values for: "high"
              date  open  high  low     close  volum

In [155]:
# Format Data
def cleanrow(row):
    namepattern = r'[A-Z]*' # To check that all tickers contains only capital letters
    datepattern = r'(\d{4})-(\d{2})-(\d{2})*' # To check that the dates are in the YYYY-MM-DD format
    matchname = re.search(namepattern, row[6])
    matchdate = re.search(datepattern, row[0])
    
    new_row = [0, 0, 0, 0, 0, 0, 0]
    new_row[0] = row[0] if bool(matchdate) == True else None 
    new_row[1] = round(float(row[1]),2) if row[1] else None
    new_row[2] = round(float(row[2]),2) if row[2] else None
    new_row[3] = round(float(row[3]),2) if row[3] else None
    new_row[4] = round(float(row[4]),2) if row[4] else None
    new_row[5] = int(row[5]) if row[5].isdigit() else None
    new_row[6] = row[6] if bool(matchname) == True else None
    # format the original data, also check for missing data and some cases of wrong data
    # missing and wrong data are replaced by None so we could deal with them more efficiently
    # in the future
    
    return new_row

def cleandata(data):
    newdata = data
    for i in range(len(data)):
        newdata[i] = cleanrow(data[i])
    return newdata

cleandata = [dataset[0]] + cleandata(data) # Cleaned data with header, for future reference


In [156]:
# Find possible wrong value
# Find missing value

def missing_value(data):
    missing_value = []
    
    for i in range(1, len(data)):
        if None in data[i]:
            missing_value.append(i)
            # find the index of rows with missing value  
        else:
            pass
        
    return missing_value

missing_value(cleandata)

[82950,
 165735,
 165858,
 205077,
 239833,
 434380,
 434503,
 478595,
 558214,
 581907,
 598237]

In [165]:
# Using webcrawler to fill in the missing data
# and check for the wrong data
import pandas_datareader as pdr
from datetime import datetime

# Deprecated webcrawler function (Not complete)
"""
def dateconvert(date):
    daysplit = date.split('-')
    year = int(daysplit[0])
    month = int(daysplit[1])
    day = int(daysplit[2])
    day2 = datetime(year, month, day)
    day1 = datetime(2013, 1, 1)
    delta = day2-day1
    # -- 2013-01-01 = 1357016400
    sfbd = (delta.days)*86400+1357016400
    return sfbd
    # This function converts the string date in the dataset into 64-bit date used by Yahoo Finance.


def YHFinance(ticker, day, category):
    url = f'https://finance.yahoo.com/quote/{ticker}/history?period1={day}&period2={day}&interval=1d&filter=history&frequency=1d'
    print(url)
    code = requests.get(url)
    plain = code.text
    s = BeautifulSoup(plain, "html.parser")
    return s
"""

def get_data(ticker, date1, date2, category):
    year1, month1, day1 = int(date1.split('-')[0]), int(date1.split('-')[1]), int(date1.split('-')[2])
    year2, month2, day2 = int(date2.split('-')[0]), int(date2.split('-')[1]), int(date2.split('-')[2])
    data = pdr.get_data_yahoo(symbols = ticker, start = datetime(year1,month1,day1), end = datetime(year2, month2, day2))
    n = data[category]
    return round(float(n[0]),2)

def get_missing(row1, row2):
    print(row1)
    for i in range(len(row1)):
        if row1[i] == None:
            category = cleandata[0][i].capitalize()
            print(category)
            datapoint = get_data(row1[6], row1[0], row2[0], category)
            print(row1[6], row1[0], row2[0], category)
            row1[i] = datapoint
        else:
            pass
    return row1

get_missing(cleandata[82950], cleandata[82951])

['2017-07-26', None, None, None, 69.08, 3, 'BHF']
Open
BHF 2017-07-26 2017-07-27 Open
High
BHF 2017-07-26 2017-07-27 High
Low
BHF 2017-07-26 2017-07-27 Low


['2017-07-26', 69.08, 69.08, 69.08, 69.08, 3, 'BHF']

In [164]:
def wrong_value(data):    
    suspicious_high = []
    suspicious_low = []
    suspicious_openclose = []
    
    for i in range(len(data)):
        if ((data[i][2] - data[i][3])/data[i][3]) >= 0.4:
            suspicious_high.append(i)
            suspicious_low.append(i)
            # find suspicious high-low gap
        elif abs((data[i][1] - data[i][4])/data[i][4]) >= 0.4:
            suspicious_openclose.append(i)
            # find suspicious open-close gap
        elif data[i][2] < data[i][1] or data[i][2] < data[i][3] or data[i][2] < data[i][4]:
            suspicious_high.append(i)
            # find low, open, close value greater than the high value
        elif data[i][3] > data[i][1] or data[i][3] > data[i][2] or data[i][3] > data[i][4]:
            suspicious_low.append(i)
            # find high, open, close value less than low value
    return suspicious_high, suspicious_low, suspicious_openclose
    
def get_high(row1, row2, cat_num = 2):
    print(row1)
    datapoint = get_data(row1[6], row1[0], row2[0], cleandata[0][cat_num].capitalize())
    row1[cat_num] = datapoint
    return row1

def get_low(row1, row2, cat_num = 3):
    datapoint = get_data(row1[6], row1[0], row2[0], cleandata[0][cat_num].capitalize())
    row1[cat_num] = datapoint
    return row1

def get_open(row1, row2, cat_num = 1):
    datapoint = get_data(row1[6], row1[0], row2[0], cleandata[0][cat_num].capitalize())
    row1[cat_num] = datapoint
    return row1

def get_close(row1, row2, cat_num = 4):
    datapoint = get_data(row1[6], row1[0], row2[0], cleandata[0][cat_num].capitalize())
    row1[cat_num] = datapoint
    return row1

get_high(cleandata[33338], cleandata[33339])

['2015-07-02', 64.66, 65.31, 63.82, 65.01, 1109120, 'ALK']


['2015-07-02', 64.66, 65.31, 63.82, 65.01, 1109120, 'ALK']

In [46]:
class Equity:
    def __init__(self, name):
        self.name = name
        self.history = [dataset[0]]+[i for i in cleandata if i[6] == name]
    
    def RoR(self, day1, day2):
        s = [i[4] for i in self.history if i[0] == day1][0]
        St = [i[4] for i in self.history if i[0] == day2][0]
        return (St/s)-1